## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Hithadhoo,MV,-0.6000,73.0833,81.95,70,99,5.88,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,87.39,54,20,8.05,few clouds
2,2,Kangaba,ML,11.9333,-8.4167,76.50,87,98,3.38,moderate rain
3,3,Mar Del Plata,AR,-38.0023,-57.5575,48.22,61,40,14.97,scattered clouds
4,4,Husavik,IS,66.0449,-17.3389,66.11,59,69,1.97,broken clouds
5,5,Abu Dhabi,AE,24.4667,54.3667,104.11,41,0,4.61,clear sky
6,6,Ushuaia,AR,-54.8000,-68.3000,49.66,53,20,6.91,few clouds
7,7,Bandarbeyla,SO,9.4942,50.8122,75.76,86,33,31.83,scattered clouds
8,8,Cantagallo,CO,7.3760,-73.9196,93.67,47,8,2.48,clear sky
9,9,Cape Town,ZA,-33.9258,18.4232,51.12,81,0,6.91,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What's the lowest temperature you would like for your trip? "))
max_temp = float(input("What's the highest temperature you would like for your trip? "))

What's the lowest temperature you would like for your trip? 65
What's the highest temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
temp_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Hithadhoo,MV,-0.6000,73.0833,81.95,70,99,5.88,overcast clouds
1,1,Jamestown,US,42.0970,-79.2353,87.39,54,20,8.05,few clouds
2,2,Kangaba,ML,11.9333,-8.4167,76.50,87,98,3.38,moderate rain
4,4,Husavik,IS,66.0449,-17.3389,66.11,59,69,1.97,broken clouds
7,7,Bandarbeyla,SO,9.4942,50.8122,75.76,86,33,31.83,scattered clouds
8,8,Cantagallo,CO,7.3760,-73.9196,93.67,47,8,2.48,clear sky
15,15,Kidal,ML,18.4411,1.4078,99.07,25,30,8.21,scattered clouds
23,23,Salalah,OM,17.0151,54.0924,77.09,94,90,5.75,overcast clouds
24,24,Carbonia,IT,39.1672,8.5222,81.84,69,1,6.38,clear sky
25,25,Bereda,ES,43.2686,-7.5406,69.58,78,0,4.34,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
temp_cities_df.count()

City_ID            490
City               490
Country            487
Lat                490
Lng                490
Max Temp           490
Humidity           490
Cloudiness         490
Wind Speed         490
Current Weather    490
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
temp_cities_df = temp_cities_df.dropna()
temp_cities_df.count()

City_ID            487
City               487
Country            487
Lat                487
Lng                487
Max Temp           487
Humidity           487
Cloudiness         487
Wind Speed         487
Current Weather    487
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temp_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Hithadhoo,MV,81.95,overcast clouds,-0.6000,73.0833,
1,Jamestown,US,87.39,few clouds,42.0970,-79.2353,
2,Kangaba,ML,76.50,moderate rain,11.9333,-8.4167,
4,Husavik,IS,66.11,broken clouds,66.0449,-17.3389,
7,Bandarbeyla,SO,75.76,scattered clouds,9.4942,50.8122,
8,Cantagallo,CO,93.67,clear sky,7.3760,-73.9196,
15,Kidal,ML,99.07,scattered clouds,18.4411,1.4078,
23,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
24,Carbonia,IT,81.84,clear sky,39.1672,8.5222,
25,Bereda,ES,69.58,clear sky,43.2686,-7.5406,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.")
        

Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel... as Jay-Z said, on to the next one.
Oh Shuckaroo, no hotel..

In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Hithadhoo,MV,81.95,overcast clouds,-0.6000,73.0833,
1,Jamestown,US,87.39,few clouds,42.0970,-79.2353,
2,Kangaba,ML,76.50,moderate rain,11.9333,-8.4167,
4,Husavik,IS,66.11,broken clouds,66.0449,-17.3389,
7,Bandarbeyla,SO,75.76,scattered clouds,9.4942,50.8122,
8,Cantagallo,CO,93.67,clear sky,7.3760,-73.9196,
15,Kidal,ML,99.07,scattered clouds,18.4411,1.4078,
23,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
24,Carbonia,IT,81.84,clear sky,39.1672,8.5222,
25,Bereda,ES,69.58,clear sky,43.2686,-7.5406,


In [9]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))